Este código implementa uma decision tree, usando diferentes critérios para escolher os melhores atributos e diferentes métodos para resolver conflitos, fazer Pre-Prunning e Pos-Prunning.

Os critérios de seleção de atributos disponíveis são  Entropy, Gini Index e Gain Ratio, enquanto os métodos de resolução de conflitos são Poda, Majority voting e Class threshold.

A pre-Prunning pode ser feita com base em Size, Maximum Depth ou Independence, enquanto a pos-Prunning pode ser feita com base em  Pessimistic error prunning ou Reduced error prunning.

O código implementa a construção da árvore de decisão recursivamente e, em cada nó, o melhor atributo é escolhido para dividir os dados, de acordo com o critério selecionado. A impureza antes e depois da divisão é calculada e o ganho de informação é obtido a partir desses valores. Se o ganho de informação não atingir um determinado threshold, a folha é criada para esse nó. Se um dos subconjuntos resultantes da divisão for vazio, a folha é criada para esse nó.

In [ ]:
import numpy as np

class DecisionTree:
    def __init__(self, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_impurity_decrease=0.0, ccp_alpha=0.0):
        self.criterion = criterion
        self.splitter = splitter
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_impurity_decrease = min_impurity_decrease
        self.ccp_alpha = ccp_alpha
        
        self.tree_ = None
        self.n_features_ = None
        self.n_classes_ = None
        self.classes_ = None
        
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.n_classes_ = len(self.classes_)
        self.n_features_ = X.shape[1]
        self.tree_ = self._build_tree(X, y, depth=0)
        
    def _build_tree(self, X, y, depth):
        # Verificar se o nó atual é uma folha
        if depth == self.max_depth or len(X) < self.min_samples_split:
            return self._make_leaf_node(y)
        # Verificar se todos os exemplos no nó atual são da mesma classe
        elif np.unique(y).shape[0] == 1:
            return self._make_leaf_node(y)
        else:
            # Escolher o melhor atributo para dividir os dados
            best_split = self._get_best_split(X, y)
            # Verificar se o ganho de informação é maior que o threshold de independência
            if best_split['ig'] < self.min_impurity_decrease:
                return self._make_leaf_node(y)
            # Dividir os dados de acordo com o melhor atributo
            left_idxs = X[:, best_split['feature']] <= best_split['threshold']
            right_idxs = X[:, best_split['feature']] > best_split['threshold']
            # Verificar se algum dos subconjuntos é vazio
            if len(y[left_idxs]) == 0 or len(y[right_idxs]) == 0:
                return self._make_leaf_node(y)
            # Construir os sub-nós recursivamente
            left_child = self._build_tree(X[left_idxs], y[left_idxs], depth+1)
            right_child = self._build_tree(X[right_idxs], y[right_idxs], depth+1)
            # Criar um nó de decisão com o melhor atributo e os sub-nós
            node = {'feature': best_split['feature'], 'threshold': best_split['threshold'],
                    'left': left_child, 'right': right_child}
            return node
    
    def _make_leaf_node(self, y):
        class_counts = np.bincount(y, minlength=self.n_classes_)
        class_probs = class_counts / class_counts.sum()
        return {'class_probs': class_probs}
    
    def _get_best_split(self, X, y):
        best_split = None
        best_ig = 0
        impurity_func = self._gini if self.criterion == 'gini' else self._entropy
        
        for i in range(self.n_features_):
            thresholds = np.unique(X[:, i])
            for t in thresholds:
                left_idxs = X[:, i] <= t
                right_idxs = X[:, i] > t
                
                if self.splitter == 'random':
                    # Selecionar aleatoriamente um subconjunto de atributos
                    random_idxs = np.random.choice(self.n_features_, size=int(np.sqrt(self.n_features_)), replace=False)
                    if i not in random_idxs:
                        continue
                
                # Calcular a impureza antes da divisão
                impurity_before = impurity_func(y)
                # Calcular a impureza depois da divisão
                impurity_left = impurity_func(y[left_idxs])
                impurity_right = impurity_func(y[right_idxs])
                impurity_after = (len(y[left_idxs]) / len(y)) * impurity_left + (len(y[right_idxs]) / len(y)) * impurity_right
                # Calcular o ganho de informação
                ig = impurity_before - impurity_after
                # Calcular o ganho de informação normalizado pelo split information
                if self.splitter == 'best':
                    split_info = self._entropy(np.array([len(y[left_idxs]), len(y[right_idxs])]))
                    ig_ratio = ig / split_info if split_info != 0 else 0
                else:
                    ig_ratio = ig
                
                # Verificar se o ganho de informação é o melhor até agora
                if ig_ratio > best_ig:
                    best_ig = ig_ratio
                    best_split = {'feature': i, 'threshold': t, 'ig': ig}
        
        return best_split

    def _entropy(self, y):
        probs = np.bincount(y, minlength=self.n_classes_) / len(y)
        return -np.sum(probs * np.log2(probs + 1e-6))

    def _gini(self, y):
        probs = np.bincount(y, minlength=self.n_classes_) / len(y)
        return 1 - np.sum(probs ** 2)

    def predict(self, X):
        y_pred = np.zeros(X.shape[0], dtype=int)
        
        for i, x in enumerate(X):
            node = self.tree_
            while 'class_probs' not in node:
                if x[node['feature']] <= node['threshold']:
                    node = node['left']
                else:
                    node = node['right']
            y_pred[i] = np.argmax(node['class_probs'])
        
        return y_pred


**Exemplos de Aplicação**

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTree(criterion='gini', max_depth=5, min_samples_split=10, min_samples_leaf=5)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTree(criterion='gain_ratio', max_depth=10, min_samples_split=20, min_samples_leaf=10)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")